In [141]:
import pandas as pd
import numpy as np
import pandas_profiling
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [139]:
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline

# Read data

In [104]:
drivers = pd.read_csv("../data/drivers.csv")
pings = pd.read_csv("../data/pings.csv")
test = pd.read_csv("../data/test.csv")

In [3]:
pings.head()

,driver_id,ping_timestamp
0,899313,1496278800
1,373017,1496278800
2,798984,1496278800
3,245966,1496278800
4,689783,1496278800


In [4]:
drivers.head()

,driver_id,gender,age,number_of_kids
0,979863,MALE,26,2
1,780123,MALE,60,2
2,614848,MALE,45,4
3,775046,MALE,62,3
4,991601,MALE,23,0


In [ ]:
test.head()

In [108]:
len(test)

17500

In [109]:
len(pings)

50528701

In [110]:
len(drivers)

2500

In [111]:
len(pings_minmax)

41676

# Transform data in pings DF

## Convert to datetime, extract date

In [32]:
pings['datetime'] = pd.to_datetime(pings.ping_timestamp, unit='s')
pings['date'] = pd.to_datetime(pings.datetime).dt.date

pandas.core.series.Series

In [34]:
pings['date'] = pd.to_datetime(pings['date'])

In [35]:
pings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50528701 entries, 0 to 50528700
Data columns (total 4 columns):
driver_id         int64
ping_timestamp    int64
datetime          datetime64[ns]
date              datetime64[ns]
dtypes: datetime64[ns](2), int64(2)
memory usage: 1.5 GB


In [36]:
pings.head()

,driver_id,ping_timestamp,datetime,date
0,899313,1496278800,2017-06-01 01:00:00,2017-06-01
1,373017,1496278800,2017-06-01 01:00:00,2017-06-01
2,798984,1496278800,2017-06-01 01:00:00,2017-06-01
3,245966,1496278800,2017-06-01 01:00:00,2017-06-01
4,689783,1496278800,2017-06-01 01:00:00,2017-06-01


## For each driver, for each day, get total number of hours spent online
- For each day, for each driver, get the min(time) and max(time) of the day.
- Example: date, driver_id, min(datetime), max(datetime)

In [37]:
# Do a group by to get the min and max timestamp by driver, day first
pings_minmax = pings.groupby(['driver_id','date'], as_index=False).agg({'datetime':[min,max]})

In [38]:
# rename columns
pings_minmax.columns = ['driver_id','date','start_time','end_time']

In [39]:
# Next, get the time difference from the min and max timestamp columns for each driver for each day
pings_minmax['time_online'] = pings_minmax['end_time'] - pings_minmax['start_time']

In [40]:
# Get in terms of hours the driver is online (this method uses numpy)
pings_minmax['hours_online_float'] = pings_minmax['time_online']/np.timedelta64(1,'h')

In [41]:
# Get in terms of hours the driver is online (this method uses pandas)
pings_minmax['hours_online_int'] = pd.to_datetime(pings_minmax['time_online']).dt.hour

In [42]:
pings_minmax.head()

,driver_id,date,start_time,end_time,time_online,hours_online_float,hours_online_int
0,111556,2017-06-01,2017-06-01 01:09:00,2017-06-01 03:22:00,02:13:00,2.216667,2
1,111556,2017-06-02,2017-06-02 01:22:00,2017-06-02 03:52:00,02:30:00,2.500000,2
2,111556,2017-06-05,2017-06-05 01:37:00,2017-06-05 06:17:00,04:40:00,4.666667,4
3,111556,2017-06-06,2017-06-06 01:51:00,2017-06-06 04:58:00,03:07:00,3.116667,3
4,111556,2017-06-07,2017-06-07 01:22:15,2017-06-07 04:00:00,02:37:45,2.629167,2


In [43]:
pings_minmax.describe()

,driver_id,time_online,hours_online_float,hours_online_int
count,41676.000000,41676,41676.000000,41676.000000
mean,560730.740810,0 days 06:03:38.641304,6.060734,5.558907
std,256787.567808,0 days 04:08:31.503043,4.142084,4.121724
min,111556.000000,0 days 00:00:00,0.000000,0.000000
25%,342310.000000,0 days 03:18:00,3.300000,3.000000
50%,562404.000000,0 days 05:40:00,5.666667,5.000000
75%,786076.000000,0 days 08:03:45,8.062500,8.000000
max,998229.000000,0 days 23:59:45,23.995833,23.000000


In [44]:
pings_minmax.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41676 entries, 0 to 41675
Data columns (total 7 columns):
driver_id             41676 non-null int64
date                  41676 non-null datetime64[ns]
start_time            41676 non-null datetime64[ns]
end_time              41676 non-null datetime64[ns]
time_online           41676 non-null timedelta64[ns]
hours_online_float    41676 non-null float64
hours_online_int      41676 non-null int64
dtypes: datetime64[ns](3), float64(1), int64(2), timedelta64[ns](1)
memory usage: 2.5 MB


# Combine the drivers and new pings into new df

In [114]:
df = pd.merge(drivers,pings_minmax,on = "driver_id")

In [115]:
df.head()

,driver_id,gender,age,number_of_kids,date,start_time,end_time,time_online,hours_online_float,hours_online_int
0,979863,MALE,26,2,2017-06-01,2017-06-01 03:25:15,2017-06-01 12:13:45,08:48:30,8.808333,8
1,979863,MALE,26,2,2017-06-02,2017-06-02 03:16:00,2017-06-02 14:57:00,11:41:00,11.683333,11
2,979863,MALE,26,2,2017-06-03,2017-06-03 03:48:00,2017-06-03 13:39:00,09:51:00,9.850000,9
3,979863,MALE,26,2,2017-06-04,2017-06-04 03:44:00,2017-06-04 12:47:00,09:03:00,9.050000,9
4,979863,MALE,26,2,2017-06-06,2017-06-06 03:31:00,2017-06-06 14:36:00,11:05:00,11.083333,11


# Check data

In [47]:
pandas_profiling.ProfileReport(df)

Number of variables,10
Number of observations,41738
Total Missing (%),0.0%
Total size in memory,3.5 MiB
Average record size in memory,88.0 B
Numeric,4
Categorical,2
Boolean,0
Date,3
Text (Unique),0
Rejected,1


Looks like there is no missing data

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41738 entries, 0 to 41737
Data columns (total 10 columns):
driver_id             41738 non-null int64
gender                41738 non-null object
age                   41738 non-null int64
number_of_kids        41738 non-null int64
date                  41738 non-null datetime64[ns]
start_time            41738 non-null datetime64[ns]
end_time              41738 non-null datetime64[ns]
time_online           41738 non-null timedelta64[ns]
hours_online_float    41738 non-null float64
hours_online_int      41738 non-null int64
dtypes: datetime64[ns](3), float64(1), int64(4), object(1), timedelta64[ns](1)
memory usage: 3.5+ MB


In [117]:
df.head()

,driver_id,gender,age,number_of_kids,date,start_time,end_time,time_online,hours_online_float,hours_online_int
0,979863,MALE,26,2,2017-06-01,2017-06-01 03:25:15,2017-06-01 12:13:45,08:48:30,8.808333,8
1,979863,MALE,26,2,2017-06-02,2017-06-02 03:16:00,2017-06-02 14:57:00,11:41:00,11.683333,11
2,979863,MALE,26,2,2017-06-03,2017-06-03 03:48:00,2017-06-03 13:39:00,09:51:00,9.850000,9
3,979863,MALE,26,2,2017-06-04,2017-06-04 03:44:00,2017-06-04 12:47:00,09:03:00,9.050000,9
4,979863,MALE,26,2,2017-06-06,2017-06-06 03:31:00,2017-06-06 14:36:00,11:05:00,11.083333,11


# Feature engineering
Since only date is expandable for feature engineering, we will use that to get more info 

In [118]:
# Lets get day of week from the date
df['day_of_week'] = df['date'].dt.dayofweek

In [119]:
#Convert date to an integer using ordinal
df['date_ord'] = df['date'].map(dt.datetime.toordinal)

In [120]:
#When you want to convert new sting time to ordinal form the test set
# d = dt.datetime.strptime("2017-06-06", "%Y-%m-%d").date()
# d.toordinal()

In [121]:
# convert gender to category
df['gender'] = df['gender'].map( {'MALE':1, 'FEMALE':0} )

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41738 entries, 0 to 41737
Data columns (total 12 columns):
driver_id             41738 non-null int64
gender                41738 non-null int64
age                   41738 non-null int64
number_of_kids        41738 non-null int64
date                  41738 non-null datetime64[ns]
start_time            41738 non-null datetime64[ns]
end_time              41738 non-null datetime64[ns]
time_online           41738 non-null timedelta64[ns]
hours_online_float    41738 non-null float64
hours_online_int      41738 non-null int64
day_of_week           41738 non-null int64
date_ord              41738 non-null int64
dtypes: datetime64[ns](3), float64(1), int64(7), timedelta64[ns](1)
memory usage: 4.1 MB


In [123]:
df.head()

,driver_id,gender,age,number_of_kids,date,start_time,end_time,time_online,hours_online_float,hours_online_int,day_of_week,date_ord
0,979863,1,26,2,2017-06-01,2017-06-01 03:25:15,2017-06-01 12:13:45,08:48:30,8.808333,8,3,736481
1,979863,1,26,2,2017-06-02,2017-06-02 03:16:00,2017-06-02 14:57:00,11:41:00,11.683333,11,4,736482
2,979863,1,26,2,2017-06-03,2017-06-03 03:48:00,2017-06-03 13:39:00,09:51:00,9.850000,9,5,736483
3,979863,1,26,2,2017-06-04,2017-06-04 03:44:00,2017-06-04 12:47:00,09:03:00,9.050000,9,6,736484
4,979863,1,26,2,2017-06-06,2017-06-06 03:31:00,2017-06-06 14:36:00,11:05:00,11.083333,11,1,736486


# Drop unecessary columns

In [124]:
df.drop(columns=["date","start_time","end_time","time_online","hours_online_float"],inplace=True)

In [125]:
df.head()

,driver_id,gender,age,number_of_kids,hours_online_int,day_of_week,date_ord
0,979863,1,26,2,8,3,736481
1,979863,1,26,2,11,4,736482
2,979863,1,26,2,9,5,736483
3,979863,1,26,2,9,6,736484
4,979863,1,26,2,11,1,736486


In [126]:
len(df)

41738

In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41738 entries, 0 to 41737
Data columns (total 7 columns):
driver_id           41738 non-null int64
gender              41738 non-null int64
age                 41738 non-null int64
number_of_kids      41738 non-null int64
hours_online_int    41738 non-null int64
day_of_week         41738 non-null int64
date_ord            41738 non-null int64
dtypes: int64(7)
memory usage: 2.5 MB


### Export df

In [150]:
df.to_csv('../data/df2.csv',index=False)

## Train test split on train data

In [130]:
features = df.loc[:, df.columns != 'hours_online_int']

target = df['hours_online_int']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size =0.3,
                                                    random_state = 7)

In [132]:
features.head()

,driver_id,gender,age,number_of_kids,day_of_week,date_ord
0,979863,1,26,2,3,736481
1,979863,1,26,2,4,736482
2,979863,1,26,2,5,736483
3,979863,1,26,2,6,736484
4,979863,1,26,2,1,736486


In [133]:
target.head()

0     8
1    11
2     9
3     9
4    11
Name: hours_online_int, dtype: int64

In [134]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# Modelling

In [136]:
from tpot import TPOTRegressor

tpot = TPOTRegressor(generations=5, population_size=20, verbosity=2)
tpot.fit(X_train, y_train) 

Generation 1 - Current best internal CV score: -10.53121546599876
Generation 2 - Current best internal CV score: -9.912086341158101
Generation 3 - Current best internal CV score: -9.912086341158101
Generation 4 - Current best internal CV score: -9.912086341158101
Generation 5 - Current best internal CV score: -9.912086341158101

Best pipeline: RandomForestRegressor(CombineDFs(input_matrix, input_matrix), bootstrap=False, max_features=0.4, min_samples_leaf=5, min_samples_split=12, n_estimators=100)


TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
       disable_update_check=False, early_stop=None, generations=5,
       max_eval_time_mins=5, max_time_mins=None, memory=None,
       mutation_rate=0.9, n_jobs=1, offspring_size=None,
       periodic_checkpoint_folder=None, population_size=20,
       random_state=None, scoring=None, subsample=1.0, use_dask=False,
       verbosity=2, warm_start=False)

In [144]:
#Tpot shows negative MSE
print("TPOT cross-validation MSE")
print(tpot.score(X_test, y_test))

TPOT cross-validation MSE
-9.38056349003882


In [145]:
#Using sklearns MSE
print('MSE:')
print(mean_squared_error(y_test, tpot.predict(X_test)))

MSE:
9.38056349003882


### Export this final tpot with pipeline

In [148]:
# NOTE: Make sure that the class is labeled 'target' in the data file
tpot.export('tpot_gojek_pipeline.py')

True